In [3]:
import json
from datetime import datetime, timedelta,date
import pandas as pd
from functools import reduce
import numpy as np 
import os.path

class ReportTM:
    
    def generate_DF(self,name_file):
        with open('data/'+name_file, encoding='utf-8') as fh:
            data = json.load(fh)

        price_date_name = 'currentPrice_'+str(name_file[:-5])
        Data_products = {
            "id" : [],
            'colorDescription' : [],
            'portrait_url' : [],
            'squarishURL' : [],
            'Stock' : [],
            'isNew' : [],
            price_date_name : [],
            'fullPrice' : [],
            'subtitle' : [],
            'title' : [],
            'date':[]
        }

        for product in data:
            Data_products['id'].append(product['id'])
            Data_products['colorDescription'].append(product['colorDescription'])
            Data_products['portrait_url'].append(product['images']['portraitURL'])
            Data_products['squarishURL'].append(product['images']['squarishURL'])
            Data_products['Stock'].append(product['inStock'])
            Data_products['isNew'].append(product['colorways'][0]['isNew'])
            Data_products[price_date_name].append(product['price']['currentPrice'])
            Data_products['fullPrice'].append(product['price']['fullPrice'])
            Data_products['subtitle'].append(product['subtitle'])
            Data_products['title'].append(product['title'])
            Data_products['date'].append(name_file[:-5])
        
        return pd.DataFrame(Data_products)

    def generate_time_serie(self,*files):
        array_dataframes =[]
        for file in files:
            with open('data/'+file, encoding='utf-8') as fh:
                data = json.load(fh)

            price_date_name = file[:-5]
            Data_products = {
                "id" : [],
                price_date_name : []

            }

            for product in data:
                Data_products['id'].append(product['id'])
                Data_products[price_date_name].append(product['price']['currentPrice'])
            array_dataframes.append(pd.DataFrame(Data_products))

        # merge a la lista de dataframes

        df_final = reduce(lambda left,right: pd.merge(left,right,on='id', how='outer'), array_dataframes)

        return df_final
    
    def check_changes_2days(self, day):
    
        #obtenemos el dia anterior a la fecha dada
        date = datetime.strptime(day, "%Y-%m-%d")
        modified_date = date - timedelta(days=1)
        prev_day = datetime.strftime(modified_date, "%Y-%m-%d")
        #comprobamos que existan archivos para los dos dias
        day_file = day +'.json'
        prev_day_file = prev_day+'.json'
        if os.path.isfile('data/'+day_file) and os.path.isfile('data/'+prev_day_file) :
            df_time = self.generate_time_serie(prev_day_file,day_file)
            df_time['change'] = df_time[prev_day_file[:-5]] - df_time[day_file[:-5]]
            return df_time.loc[df_time['change']>0]
            #
        else:
            raise ValueError(
                    "Nombres de archivo no validos. "
                        f"Only one file per ZIP: {prev_day_file,day_file}"
                    )
    

In [51]:
import smtplib
import imghdr
from email.message import EmailMessage
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from jinja2 import Environment  
EMAIL_ADDRESS = 'seiitoblox@gmail.com'
EMAIL_PASSWORD = 'Zero1281'
	
    
data = [{'url_img': 'https://ibb.co/9r5GCtX',
  'title': 'Nike Air Max 95',
  'prev': 3349.0,
  'actual': 3199.0,
  'url': 'https://ibb.co/9r5GCtX'}
        ,{
    'url_img':'https://static.nike.com/a/images/c_limit,w_400,f_auto/t_product_v1/b71918af-c3b5-4c14-ac35-6f9e8b80c9c7/image.jpg',
    'title':'Nike phantom',
    'prev':100,
    'actual':100,
    'url': 'https://www.nike.com/mx/t/calzado-air-max-270-react-2LDsVK/CT1264-103'
},
{
    'url_img':'https://static.nike.com/a/images/c_limit,w_400,f_auto/t_product_v1/b71918af-c3b5-4c14-ac35-6f9e8b80c9c7/image.jpg',
    'title':'Nike phantom',
    'prev':100,
    'actual':100,
    'url': 'https://www.nike.com/mx/t/calzado-air-max-270-react-2LDsVK/CT1264-103'
},{
    'url_img':'https://static.nike.com/a/images/c_limit,w_400,f_auto/t_product_v1/b71918af-c3b5-4c14-ac35-6f9e8b80c9c7/image.jpg',
    'title':'Nike phantom',
    'prev':100,
    'actual':100,
    'url': 'https://www.nike.com/mx/t/calzado-air-max-270-react-2LDsVK/CT1264-103'
}]

HTML = """\

<!doctype html><html xmlns="http://www.w3.org/1999/xhtml" xmlns:v="urn:schemas-microsoft-com:vml" xmlns:o="urn:schemas-microsoft-com:office:office"><head> <title> </title> <!--[if !mso]><!-- --> <meta http-equiv="X-UA-Compatible" content="IE=edge"> <!--<![endif]--> <meta http-equiv="Content-Type" content="text/html; charset=UTF-8"> <meta name="viewport" content="width=device-width, initial-scale=1"> <style type="text/css"> #outlook a { padding: 0; } body { margin: 0; padding: 0; -webkit-text-size-adjust: 100%; -ms-text-size-adjust: 100%; } table, td { border-collapse: collapse; mso-table-lspace: 0pt; mso-table-rspace: 0pt; } img { border: 0; height: auto; line-height: 100%; outline: none; text-decoration: none; -ms-interpolation-mode: bicubic; } p { display: block; margin: 13px 0; } </style> <!--[if mso]> <xml> <o:OfficeDocumentSettings> <o:AllowPNG/> <o:PixelsPerInch>96</o:PixelsPerInch> </o:OfficeDocumentSettings> </xml> <![endif]--> <!--[if lte mso 11]> <style type="text/css"> .mj-outlook-group-fix { width:100% !important; } </style> <![endif]--> <!--[if !mso]><!--> <link href="https://fonts.googleapis.com/css?family=Ubuntu:300,400,500,700" rel="stylesheet" type="text/css"> <style type="text/css"> @import url(https://fonts.googleapis.com/css?family=Ubuntu:300,400,500,700); </style> <!--<![endif]--> <style type="text/css"> @media only screen and (min-width:480px) { .mj-column-per-50 { width: 50% !important; max-width: 50%; } .mj-column-per-33-333333333333336 { width: 33.333333333333336% !important; max-width: 33.333333333333336%; } } </style> <style type="text/css"> @media only screen and (max-width:480px) { table.mj-full-width-mobile { width: 100% !important; } td.mj-full-width-mobile { width: auto !important; } } </style></head><body style="background-color:#d6dde5;"> <div style="background-color:#d6dde5;"> <!--[if mso | IE]> <table align="center" border="0" cellpadding="0" cellspacing="0" class="" style="width:600px;" width="600" > <tr> <td style="line-height:0px;font-size:0px;mso-line-height-rule:exactly;"> <![endif]--> <div style="background:#ffffff;background-color:#ffffff;margin:0px auto;max-width:600px;"> <table align="center" border="0" cellpadding="0" cellspacing="0" role="presentation" style="background:#ffffff;background-color:#ffffff;width:100%;"> <tbody> <tr> <td style="direction:ltr;font-size:0px;padding:20px 0;text-align:center;"> <!--[if mso | IE]> <table role="presentation" border="0" cellpadding="0" cellspacing="0"> <tr> <td class="" style="vertical-align:top;width:300px;" > <![endif]--> <div class="mj-column-per-50 mj-outlook-group-fix" style="font-size:0px;text-align:left;direction:ltr;display:inline-block;vertical-align:top;width:100%;"> <table border="0" cellpadding="0" cellspacing="0" role="presentation" style="vertical-align:top;" width="100%"> <tr> <td align="center" style="font-size:0px;padding:10px;word-break:break-word;"> <table border="0" cellpadding="0" cellspacing="0" role="presentation" style="border-collapse:collapse;border-spacing:0px;"> <tbody> <tr> <td style="width:280px;"> <a href="https://mjml.io" target="_blank"> <img alt="Mosquitar" height="auto" src="https://i.ibb.co/b5nZLzC/Logo-spy.png" style="border:0;display:block;outline:none;text-decoration:none;height:auto;width:100%;font-size:13px;" width="280" /> </a> </td> </tr> </tbody> </table> </td> </tr> </table> </div> <!--[if mso | IE]> </td> <td class="" style="vertical-align:top;width:300px;" > <![endif]--> <div class="mj-column-per-50 mj-outlook-group-fix" style="font-size:0px;text-align:left;direction:ltr;display:inline-block;vertical-align:top;width:100%;"> <table border="0" cellpadding="0" cellspacing="0" role="presentation" style="vertical-align:top;" width="100%"> </table> </div> <!--[if mso | IE]> </td> </tr> </table> <![endif]--> </td> </tr> </tbody> </table> </div> <!--[if mso | IE]> </td> </tr> </table> <table align="center" border="0" cellpadding="0" cellspacing="0" class="" style="width:600px;" width="600" > <tr> <td style="line-height:0px;font-size:0px;mso-line-height-rule:exactly;"> <![endif]--> <!--[if mso | IE]> </td> </tr> </table> <table align="center" border="0" cellpadding="0" cellspacing="0" class="" style="width:600px;" width="600" > <tr> <td style="line-height:0px;font-size:0px;mso-line-height-rule:exactly;"> <![endif]--> <div style="background:#ffffff;background-color:#ffffff;margin:0px auto;max-width:600px;"> <table align="center" border="0" cellpadding="0" cellspacing="0" role="presentation" style="background:#ffffff;background-color:#ffffff;width:100%;"> <tbody> <tr> <td style="direction:ltr;font-size:0px;padding:20px 0;padding-bottom:20px;text-align:center;"> <!--[if mso | IE]> <table role="presentation" border="0" cellpadding="0" cellspacing="0"> <tr> <td class="" style="vertical-align:bottom;width:300px;" > <![endif]--> <!--[if mso | IE]> </td> <td class="" style="vertical-align:bottom;width:300px;" > <![endif]--> {% for product in products -%} <div class="mj-column-per-50 mj-outlook-group-fix" style="font-size:0px;text-align:left;direction:ltr;display:inline-block;vertical-align:bottom;width:100%;"> <table border="0" cellpadding="0" cellspacing="0" role="presentation" style="vertical-align:bottom;" width="100%"> <tr> <td align="center" style="font-size:0px;padding:10px 25px;padding-top:20px;padding-right:0px;padding-bottom:20px;padding-left:0px;word-break:break-word;"> <table border="0" cellpadding="0" cellspacing="0" role="presentation" style="border-collapse:collapse;border-spacing:0px;"> <tbody> <tr> <td style="width:208px;"> <img alt="Blake polo shirt" height="auto" src={{ product.url_img }} style="border:none;display:block;outline:none;text-decoration:none;height:auto;width:100%;font-size:13px;" width="208" /> </td> </tr> </tbody> </table> </td> </tr> <tr> <td align="center" style="font-size:0px;padding:10px 25px;padding-top:0;padding-right:25px;padding-bottom:0px;padding-left:25px;word-break:break-word;"> <div style="font-family:Ubuntu, Helvetica, Arial, sans-serif;font-size:15px;font-weight:bold;line-height:1;text-align:center;color:#000000;"> <p>{{ product.title }}</p> <p style="font-weight:normal"><s>{{ product.prev }}&#36;</s></p> <p style="font-weight:normal">{{ product.actual }}&#36;</p> </div> </td> </tr> <tr> <td align="center" vertical-align="middle" style="font-size:0px;padding:15px 30px;padding-top:0;padding-bottom:10px;word-break:break-word;"> <table border="0" cellpadding="0" cellspacing="0" role="presentation" style="border-collapse:separate;line-height:100%;"> <tr> <td align="center" bgcolor="#000000" role="presentation" style="border:none;border-radius:3px;cursor:auto;mso-padding-alt:10px 25px;background:#000000;" valign="middle"> <a href="{{ product.url }}" style="display:inline-block;background:#000000;color:#ffffff;font-family:Ubuntu, Helvetica, Arial, sans-serif;font-size:13px;font-weight:normal;line-height:120%;margin:0;text-decoration:none;text-transform:none;padding:10px 25px;mso-padding-alt:0px;border-radius:3px;" target="_blank"> COMPRAR </a> </td> </tr> </table> </td> </tr> </table> </div> {% endfor %} <!--[if mso | IE]> </td> </tr> </table> <![endif]--> </td> </tr> </tbody> </table> </div> <!--[if mso | IE]> </td> </tr> </table> <table align="center" border="0" cellpadding="0" cellspacing="0" class="" style="width:600px;" width="600" > <tr> <td style="line-height:0px;font-size:0px;mso-line-height-rule:exactly;"> <![endif]--> <div style="background:#fa8739;background-color:#fa8739;margin:0px auto;max-width:600px; display: none;"> <table align="center" border="0" cellpadding="0" cellspacing="0" role="presentation" style="background:#fa8739;background-color:#fa8739;width:100%;"> <tbody> <tr> <td style="direction:ltr;font-size:0px;padding:20px 0;padding-bottom:20px;padding-top:0;text-align:center;"> <!--[if mso | IE]> <table role="presentation" border="0" cellpadding="0" cellspacing="0"> <tr> <td class="" style="vertical-align:top;width:200px;" > <![endif]--> <div class="mj-column-per-33-333333333333336 mj-outlook-group-fix" style="font-size:0px;text-align:left;direction:ltr;display:inline-block;vertical-align:top;width:100%;"> <table border="0" cellpadding="0" cellspacing="0" role="presentation" style="vertical-align:top;" width="100%"> <tr> <td align="center" style="font-size:0px;padding:10px 25px;padding-top:10px;padding-right:0px;padding-bottom:0px;padding-left:0px;word-break:break-word;"> <table border="0" cellpadding="0" cellspacing="0" role="presentation" style="border-collapse:collapse;border-spacing:0px;"> <tbody> <tr> <td style="width:180px;"> <img alt="Mosquitar" height="auto" src="http://191n.mj.am/img/191n/3s/x49.png" style="border:none;display:block;outline:none;text-decoration:none;height:auto;width:100%;font-size:13px;" width="180" /> </td> </tr> </tbody> </table> </td> </tr> </table> </div> <!--[if mso | IE]> </td> <td class="" style="vertical-align:top;width:200px;" > <![endif]--> <div class="mj-column-per-33-333333333333336 mj-outlook-group-fix" style="font-size:0px;text-align:left;direction:ltr;display:inline-block;vertical-align:top;width:100%;"> <table border="0" cellpadding="0" cellspacing="0" role="presentation" style="vertical-align:top;" width="100%"> <tr> <td align="center" style="font-size:0px;padding:10px 25px;padding-top:0;padding-right:25px;padding-bottom:0px;padding-left:25px;word-break:break-word;"> <div style="font-family:Ubuntu, Helvetica, Arial, sans-serif;font-size:13px;line-height:1;text-align:center;color:#ffffff;"> <p><a href="https://mjml.io" style="text-decoration: none; color: inherit;">Privacy policy</a></p> </div> </td> </tr> </table> </div> <!--[if mso | IE]> </td> <td class="" style="vertical-align:top;width:200px;" > <![endif]--> <div class="mj-column-per-33-333333333333336 mj-outlook-group-fix" style="font-size:0px;text-align:left;direction:ltr;display:inline-block;vertical-align:top;width:100%;"> <table border="0" cellpadding="0" cellspacing="0" role="presentation" style="vertical-align:top;" width="100%"> <tr> <td align="center" style="font-size:0px;padding:10px 25px;padding-top:20px;word-break:break-word;"> <!--[if mso | IE]> <table align="center" border="0" cellpadding="0" cellspacing="0" role="presentation" > <tr> <td> <![endif]--> <table align="center" border="0" cellpadding="0" cellspacing="0" role="presentation" style="float:none;display:inline-table;"> <tr> <td style="padding:4px;"> <table border="0" cellpadding="0" cellspacing="0" role="presentation" style="background:#3b5998;border-radius:3px;width:20px;"> <tr> <td style="font-size:0;height:20px;vertical-align:middle;width:20px;"> <img height="20" src="https://www.mailjet.com/images/theme/v1/icons/ico-social/facebook.png" style="border-radius:3px;display:block;" width="20" /> </td> </tr> </table> </td> </tr> </table> <!--[if mso | IE]> </td> <td> <![endif]--> <table align="center" border="0" cellpadding="0" cellspacing="0" role="presentation" style="float:none;display:inline-table;"> <tr> <td style="padding:4px;"> <table border="0" cellpadding="0" cellspacing="0" role="presentation" style="background:#55acee;border-radius:3px;width:20px;"> <tr> <td style="font-size:0;height:20px;vertical-align:middle;width:20px;"> <img height="20" src="https://www.mailjet.com/images/theme/v1/icons/ico-social/twitter.png" style="border-radius:3px;display:block;" width="20" /> </td> </tr> </table> </td> </tr> </table> <!--[if mso | IE]> </td> <td> <![endif]--> <table align="center" border="0" cellpadding="0" cellspacing="0" role="presentation" style="float:none;display:inline-table;"> <tr> <td style="padding:4px;"> <table border="0" cellpadding="0" cellspacing="0" role="presentation" style="background:#dc4e41;border-radius:3px;width:20px;"> <tr> <td style="font-size:0;height:20px;vertical-align:middle;width:20px;"> <img height="20" src="https://www.mailjet.com/images/theme/v1/icons/ico-social/google-plus.png" style="border-radius:3px;display:block;" width="20" /> </td> </tr> </table> </td> </tr> </table> <!--[if mso | IE]> </td> </tr> </table> <![endif]--> </td> </tr> </table> </div> <!--[if mso | IE]> </td> </tr> </table> <![endif]--> </td> </tr> </tbody> </table> </div> <!--[if mso | IE]> </td> </tr> </table> <table align="center" border="0" cellpadding="0" cellspacing="0" class="" style="width:600px;" width="600" > <tr> <td style="line-height:0px;font-size:0px;mso-line-height-rule:exactly;"> <![endif]--> <div style="margin:0px auto;max-width:600px;"> <table align="center" border="0" cellpadding="0" cellspacing="0" role="presentation" style="width:100%;"> <tbody> <tr> <td style="direction:ltr;font-size:0px;padding:20px 0;padding-bottom:20px;padding-top:0px;text-align:center;"> <!--[if mso | IE]> <table role="presentation" border="0" cellpadding="0" cellspacing="0"> <tr> </tr> </table> <![endif]--> </td> </tr> </tbody> </table> </div> <!--[if mso | IE]> </td> </tr> </table> <![endif]--> </div></body></html>

"""
msgs = MIMEText(
    Environment().from_string(HTML).render(
        title='Hello World!',
        products=final_array[0:5]
    ), "html"
)

msg = EmailMessage()
msg['Subject'] = 'Nike crawler Info'
msg['From'] = EMAIL_ADDRESS
msg['To'] = ['saga_cth@hotmail.com','stiksnaki@gmail.com']
    
msg.set_content('This is plain text email content..')
msg.add_alternative(msgs.as_string(), subtype='html')
with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp:
    smtp.login(EMAIL_ADDRESS, EMAIL_PASSWORD)
    smtp.send_message(msg)

In [6]:
info =ReportTM()


In [7]:
kk = info.check_changes_2days('2020-07-02')
kk

,id,2020-07-01,2020-07-02,change
1,228e4770-dc0e-3dc9-a9f9-05ead31aea5a,3349.0,3199.0,150.0
30,7df6d230-516d-3279-a6c4-ef2a36197b0f,2499.0,2399.0,100.0
150,1c2abe09-344e-3f17-963e-f9dbe896190f,4199.0,3999.0,200.0
219,23d48384-802a-3125-9ceb-3c2f4313d119,3349.0,3199.0,150.0
224,6d3971d4-8538-3037-a2b3-7ccf8408dbd8,1999.0,1799.0,200.0
256,c2ba4bb1-09fb-36d8-976b-6d895d1d04b8,4499.0,4199.0,300.0
257,2cbd5eff-87ad-3a37-9442-cb84cbe1b172,2099.0,1999.0,100.0
258,b8308f16-caf1-3ada-b153-df332bccab63,3299.0,3149.0,150.0


In [8]:

def take_info(path, id_):
    df = pd.read_csv(path)
    element = df.loc[df.id == id_]
    return element.to_dict(orient='records')
    

In [9]:
day= '2020-07-02'
prev_day = '2020-07-01'
final_array = []
for element in kk.to_dict(orient='records'):
    serve_dicto = {
    'url_img':'',
    'title':'',
    'prev':'',
    'actual':'',
    'url':''
    }
    extra_info = take_info('data/dic_products/products.csv',element['id'])
    serve_dicto['prev'] =element[prev_day]
    serve_dicto['actual']=element[day]
    serve_dicto['title'] = extra_info[0]['title']
    serve_dicto['url_img'] = extra_info[0]['portrait_url']
    serve_dicto['url'] = 'https://www.nike.com/mx'+extra_info[0]['url'][13:]
    print(serve_dicto)
    final_array.append(serve_dicto)
    
    

{'url_img': 'https://c.static-nike.com/a/images/ar_4:5,c_pad,w_400,f_auto/t_product_v1/cmuof8adhfipkvd0f43r/calzado-air-max-95-PATZx9Gr.jpg', 'title': 'Nike Air Max 95', 'prev': 3349.0, 'actual': 3199.0, 'url': 'https://www.nike.com/mx/t/calzado-air-max-95-PATZx9Gr/609048-109'}
{'url_img': 'https://static.nike.com/a/images/c_limit,w_400,f_auto/t_product_v1/a164a629-1353-4f02-85e5-62fed505f738/image.jpg', 'title': 'Nike SuperRep Cycle', 'prev': 2499.0, 'actual': 2399.0, 'url': 'https://www.nike.com/mx/t/calzado-de-ciclismo-indoor-superrep-cycle-Fm3V6v/CW2191-008'}
{'url_img': 'https://static.nike.com/a/images/c_limit,w_400,f_auto/t_product_v1/d1d586ba-9202-4c41-ba05-85d5e5e861cd/image.jpg', 'title': 'Nike Romaleos 4', 'prev': 4199.0, 'actual': 3999.0, 'url': 'https://www.nike.com/mx/t/calzado-de-entrenamiento-romaleos-4-f1nCw3/CD3463-010'}
{'url_img': 'https://static.nike.com/a/images/c_limit,w_400,f_auto/t_product_v1/i1-0cf2c7c9-04f0-4c94-8435-a1a8cd4603ba/image.jpg', 'title': 'Nike Ai

In [10]:
take_info('data/dic_products/products.csv','228e4770-dc0e-3dc9-a9f9-05ead31aea5a')[0]['title']

'Nike Air Max 95'

In [11]:
final_array[0:5]

[{'url_img': 'https://c.static-nike.com/a/images/ar_4:5,c_pad,w_400,f_auto/t_product_v1/cmuof8adhfipkvd0f43r/calzado-air-max-95-PATZx9Gr.jpg',
  'title': 'Nike Air Max 95',
  'prev': 3349.0,
  'actual': 3199.0,
  'url': 'https://www.nike.com/mx/t/calzado-air-max-95-PATZx9Gr/609048-109'},
 {'url_img': 'https://static.nike.com/a/images/c_limit,w_400,f_auto/t_product_v1/a164a629-1353-4f02-85e5-62fed505f738/image.jpg',
  'title': 'Nike SuperRep Cycle',
  'prev': 2499.0,
  'actual': 2399.0,
  'url': 'https://www.nike.com/mx/t/calzado-de-ciclismo-indoor-superrep-cycle-Fm3V6v/CW2191-008'},
 {'url_img': 'https://static.nike.com/a/images/c_limit,w_400,f_auto/t_product_v1/d1d586ba-9202-4c41-ba05-85d5e5e861cd/image.jpg',
  'title': 'Nike Romaleos 4',
  'prev': 4199.0,
  'actual': 3999.0,
  'url': 'https://www.nike.com/mx/t/calzado-de-entrenamiento-romaleos-4-f1nCw3/CD3463-010'},
 {'url_img': 'https://static.nike.com/a/images/c_limit,w_400,f_auto/t_product_v1/i1-0cf2c7c9-04f0-4c94-8435-a1a8cd4603b